## Imports

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd

## Read Data

In [2]:
arrivals_df = pd.read_excel("Arrivals.xlsx")
arrivals_df["Type"] = "Arrival"

departures_df = pd.read_excel("Departures.xlsx")
departures_df["Type"] = "Departure"

gates_df = pd.read_excel("Gates.xlsx")

all_flights_df = pd.concat([arrivals_df, departures_df])

## Utility Functions

In [3]:
def get_distance(i): # Distance of gate i to gate 0
    return gates_df[0][i]

def get_arrival_time(i):
    return all_flights_df[all_flights_df["Flight Number"] == i]["Arrival Time"]

def get_departure_time(i):
    return all_flights_df[all_flights_df["Flight Number"] == i]["Departure Time"]

def get_num_passengers(i):
    return all_flights_df[all_flights_df["Flight Number"] == i]["Passengers"]

In [11]:
## Basic Assertions on the Data
flight_numbers = sorted(list(arrivals_df["Flight Number"]) + list(departures_df["Flight Number"]))
assert(flight_numbers == list(range(1,131)))

flight_numbers = sorted(list(all_flights_df["Flight Number"]))
assert(flight_numbers == list(range(1,131)))

## Setup Gates

In [12]:
num_gates = 14
gates = [i for i in range(num_gates)]
gates

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

## Design Variables

In [13]:
num_flights = 5 # len(flight_numbers)
num_gates = len(gates)
x = cp.Variable((num_flights, len(gates)), boolean=True)

## Constraint 1 (Every aircraft must be assigned to one and only one gate.)

In [14]:
# Every aircraft must be assigned to one and only one gate.
constraint_1_list = []
for i, flight_i_variables in enumerate(x):
    constraint_1_list.append(cp.sum(x[i]) == 1)

## Constraint 2 (No two aircrafts can be assigned to the same gate at the same time.)

In [29]:
# No two aircrafts can be assigned to the same gate at the same time.
constraint_2_list = []
for i, x_i in enumerate(x):
    ai, di = get_arrival_time(i + 1), get_departure_time(i + 1)
    for j, x_j in enumerate(x):
        if i <= j:
            continue
        aj, dj = get_arrival_time(j + 1), get_departure_time(j + 1)
        for k in range(num_gates):
            constraint = (x_i[k]*x_j[k]*(dj - ai)*(di-aj) <= 0)
            constraint_2_list.append(x_i[k]*x_j[k]*(dj - ai)*(di-aj) <= 0)

/Users/vivekshankar/opt/anaconda3/envs/aa222/lib/python3.8/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 141 times so far.

  warnings.warn(msg, UserWarning)
/Users/vivekshankar/opt/anaconda3/envs/aa222/lib/python3.8/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 142 times

/Users/vivekshankar/opt/anaconda3/envs/aa222/lib/python3.8/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 278 times so far.

  warnings.warn(msg, UserWarning)
/Users/vivekshankar/opt/anaconda3/envs/aa222/lib/python3.8/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 279 times

In [18]:
len(constraint_2_list)

140

## Define Objective Function

In [20]:
print(x.shape)

(5, 14)


In [21]:
W = np.zeros(x.shape)
P = np.zeros(x.shape)

In [22]:
for i in range(W.shape[0]): # iterate over flights
    for k in range(W.shape[1]): # iterate over gates
        W[i][k] = get_distance(k+1) # Gates start from 1
        P[i][k] = get_num_passengers(i+1) # Flights start from 1

In [23]:
objective = cp.sum(cp.multiply(cp.multiply(x, W), P))

In [33]:
# Note: constraint_2_list is a list of non-linear constraints!
all_constraints = constraint_1_list# + constraint_2_list
problem = cp.Problem(cp.Minimize(objective), all_constraints)

## Solve Problem

In [34]:
result = problem.solve(solver=cp.GLPK_MI)

In [36]:
result

2235.0

In [37]:
x.value

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])